In [ ]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_validate
import warnings
warnings.filterwarnings("ignore")
import os

#read data, 2019 Data seems legit
data = pd.read_csv(r'/kaggle/input/world-happiness/2019.csv')

we havee a data set as 156 x 9. The "Overal Rank" feature is not neccessary so we will drop it. Similarly, Country region will be droped too.

Before the EDA it would be nice to clear that for the Prediction Method, output is the Score which is happiness score of the country. And the remainin 6 features are the input. It can be seen as all the inputs are numeric and output is numeric too. Hence, the problem will lead to regression task.

In [ ]:
print("shape:", data.shape)
data.head(3)

In [ ]:
print("original shape is: ", data.shape)
data.drop(['Overall rank'],axis=1,inplace=True)
print("new shape is :", data.shape)

In [ ]:
country = data.iloc[:,0:1]
country.tail()

In [ ]:
from sklearn import preprocessing
ohe= preprocessing.OneHotEncoder()
country_hot = ohe.fit_transform(country).toarray()
dfcountry =pd.DataFrame(data=country_hot, index=range(country.shape[0]))
dfcountry.head()

There are 155 countries, so that adding it by one hot encoder is not a good idea. Maybe it can be applied as a column in other words single encoder. Im sure that country as a geography affects the happiness (i know, there are parameters in data frame gives the characteristics of the country but still there is sg called fate ^^) so that "for now" i will drop it, at the end we can check the country affect.

In [ ]:
data.drop(['Country or region'],axis=1,inplace=True)
data.head(2)

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

# EDA

In [ ]:
korelasyon=data.corr()
figure, axis=plt.subplots(figsize=(10,10))
sns.heatmap(korelasyon, annot=True)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
sns.pairplot(data, hue="Score")

In [ ]:
sns.distplot(data['Score'])

In [ ]:
fig = plt.figure(figsize=(10,7))

sns.distplot(data['GDP per capita'],label="GDP per capita")
sns.distplot(data['Social support'])
sns.distplot(data['Healthy life expectancy'])
fig.legend(labels=['GDP per capita','Social support','Healthy life expectancy'])

In [ ]:
fig = plt.figure(figsize=(10,7))
sns.distplot(data['Freedom to make life choices'])
sns.distplot(data['Generosity'])
sns.distplot(data['Perceptions of corruption'])
fig.legend(labels=['Freedom to make life choices','Generosity','Perceptions of corruption'])

Everything seems fine, not to much correlation. Hence, multi-collinearity problem is seemly not exist.

# Prediction Models - Machine Learning

In [ ]:
y=data.iloc[:,0:1]
x=data.iloc[:,1:]
x.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3, random_state=12)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
RTD = DecisionTreeRegressor(random_state = 0)
RTD.fit(x_train, y_train)
y_pred = RTD.predict(x_test)
r2_score(y_test, y_pred)


In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

reg.fit(x_train,y_train)

y_pred = reg.predict(x_test)

r2_score(y_test, y_pred)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

RFR = RandomForestRegressor(n_estimators=10,random_state = 0 )
RFR.fit(x_train, y_train)
y_pred=RFR.predict(x_test)
r2_score(y_test, y_pred)

In [ ]:
from sklearn.svm import SVR

SVR_Reg = SVR(kernel = "rbf",degree=3, C=40)
SVR_Reg.fit(x_train, y_train)
y_pred = SVR_Reg.predict(x_test)
r2_score(y_test, y_pred)